## Обнаружение статистически значимых отличий в уровнях экспрессии генов больных раком

In [1]:
# import needed libraries
import pandas as pd
import numpy as np

import scipy

from statsmodels.stats.proportion import proportion_confint
import statsmodels.stats.multitest as smm

### Описание используемых данных

Данные для этой задачи взяты из исследования, проведенного в Stanford School of Medicine. В исследовании была предпринята попытка выявить набор генов, которые позволили бы более точно диагностировать возникновение рака груди на самых ранних стадиях.

В эксперименте принимали участие 24 человек, у которых не было рака груди (normal), 25 человек, у которых это заболевание было диагностировано на ранней стадии (early neoplasia), и 23 человека с сильно выраженными симптомами (cancer).

Ученые провели секвенирование биологического материала испытуемых, чтобы понять, какие из этих генов наиболее активны в клетках больных людей.

Секвенирование — это определение степени активности генов в анализируемом образце с помощью подсчёта количества соответствующей каждому гену РНК.

В данных есть именно эта количественная мера активности каждого из 15748 генов у каждого из 72 человек, принимавших участие в эксперименте.

Нужно определить те гены, активность которых у людей в разных стадиях заболевания отличается статистически значимо.

Кроме того, нужно оценить не только статистическую, но и практическую значимость этих результатов, которая часто используется в подобных исследованиях.

Диагноз человека содержится в столбце под названием "Diagnosis".

In [2]:
# read the data and take a glimpse at how it looks
data = pd.read_csv("gene_high_throughput_sequencing.csv")
data.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [3]:
data.Diagnosis.value_counts()

early neoplasia    25
normal             24
cancer             23
Name: Diagnosis, dtype: int64

### Практическая значимость изменения

Цель исследований — найти гены, средняя экспрессия которых отличается не только статистически значимо, но и достаточно сильно. В экспрессионных исследованиях для этого часто используется метрика, которая называется fold change (кратность изменения). Определяется она следующим образом:

$$
F_c(C, T) =
\begin{cases}
\frac{T}{C} , & T>C \\
-\frac{C}{T}, & T<C \\
\end{cases}
$$

где C,T C, T C,T — средние значения экспрессии гена в control и treatment группах соответственно. По сути, fold change показывает, во сколько раз отличаются средние двух выборок.

### Часть 1: применение t-критерия Стьюдента

В первой части применим критерий Стьюдента для проверки гипотезы о равенстве средних в двух независимых выборках. Применить критерий для каждого гена нужно будет дважды:

* для групп **normal (control)** и **early neoplasia (treatment)**
* для групп **early neoplasia (control)** и **cancer (treatment)**

Уровень значимости принимаем равным 0.05.

In [180]:
# Split the data in three dataframes based on diagnosis
normal = data[data.Diagnosis == 'normal']
early_neoplasia = data[data.Diagnosis == 'early neoplasia']
cancer = data[data.Diagnosis == 'cancer']

In [181]:
%%time
ttest1 = [scipy.stats.ttest_ind(normal[column], early_neoplasia[column], 
                               equal_var=False)
         for column in normal.iloc[:, 2:].columns.values]

Wall time: 6.72 s


В качестве ответа в этой части задания необходимо указать количество статистически значимых отличий, которые вы нашли с помощью t-критерия Стьюдента, то есть число генов, у которых p-value этого теста оказался меньше, чем уровень значимости. 

In [203]:
answer1 = len([test.pvalue for test in ttest1 if test.pvalue < 0.05])
print("Answer", answer1)

Answer 1575


In [183]:
def write_ans(text, answer_number=1):
    answer_number = answer_number
    with open("ans{}.txt".format(answer_number), "w") as file:
        file.write(str(text))

In [184]:
write_ans(answer1)

In [185]:
%%time
ttest2 = [scipy.stats.ttest_ind(early_neoplasia[column], cancer[column], 
                               equal_var=False)
         for column in early_neoplasia.iloc[:, 2:].columns.values]

Wall time: 6 s


In [200]:
answer2 = len([test.pvalue for test in ttest2 if test.pvalue < 0.05])
print("Answer",answer2)

Answer 3490


In [187]:
write_ans(answer2, answer_number=2)

### Часть 2: поправка методом Холма

В этой части применим поправку Холма для получившихся двух наборов достигаемых уровней значимости из предыдущей части. Поскольку мы будем делать поправку для каждого из двух наборов p-value отдельно, то проблема, связанная с множественной проверкой останется.

Для того, чтобы ее устранить, достаточно воспользоваться поправкой Бонферрони, то есть использовать уровень значимости 0.05 / 2 вместо 0.05 для дальнейшего уточнения значений p-value c помощью метода Холма.

In [188]:
# Start with normal vs early neoplasia
pvalues1 = [test.pvalue for test in ttest1]
reject1, p_corrected1, a1, a2 = smm.multipletests(pvalues1,
                                                  alpha=0.05/2,
                                                  method='holm')

In [191]:
mask = np.array(p_corrected1) < 0.025

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Холма-Бонферрони. Причем это число нужно ввести с учетом практической значимости: посчитайте для каждого значимого изменения **fold change** и выпишите в ответ число таких значимых изменений, абсолютное значение **fold change** которых больше, чем $1.5$.

При использовании поправки на уровне значимости 0.025 меняются значения достигаемого уровня значимости, но не меняется значение уровня доверия (то есть для отбора значимых изменений скорректированные значения уровня значимости нужно сравнивать с порогом 0.025, а не 0.05)! 

In [192]:
# Define the fold change function.
# It will take two array of mean values as an input.

def fold_change(T, C):
    df = pd.DataFrame([T, C])
    fold_change = df.apply(lambda x: x[0]/x[1].astype(float) if x[0]
                           > x[1] else -x[1]/x[0].astype(float))
    return fold_change

In [193]:
normal_mean = normal.iloc[:, 2:].apply(np.mean, axis=0)
early_neoplasia_mean = early_neoplasia.iloc[:, 2:].apply(np.mean, axis=0)

In [201]:
fold_changes = fold_change(normal_mean[mask], early_neoplasia_mean[mask])
ans3 = sum(abs(fold_changes) > 1.5)
print("Answer",ans3)

Answer 2


In [198]:
write_ans(ans3, answer_number=3)

#### Проверка начимых отличий по группам early neoplasia и cancer

In [212]:
# Let's do the same for early_neoplasia vs cancer
pvalues2 = [test.pvalue for test in ttest2]
reject, p_corrected2, a1, a2 = smm.multipletests(pvalues2,
                                                 alpha=0.025,
                                                 method='holm')

In [213]:
mask = np.array(p_corrected2) < 0.025

In [214]:
cancer_mean = cancer.iloc[:, 2:].apply(np.mean, axis=0)

In [215]:
fold_changes = fold_change(early_neoplasia_mean[mask], cancer_mean[mask])

ans4 = sum(abs(fold_changes) > 1.5)

print("Answer", ans4)

Answer 77


In [211]:
write_ans(ans4, answer_number=4)

### Часть 3: поправка методом Бенджамини-Хохберга

Данная часть задания аналогична второй части за исключением того, что будет использован метод Бенджамини-Хохберга.

Методы коррекции, которые контролируют FDR, допускают больше ошибок первого рода и имеют большую мощность, чем методы, контролирующие FWER. Большая мощность означает, что эти методы будут совершать меньше ошибок второго рода (то есть будут лучше улавливать отклонения от $H_0$, когда они есть, и будут чаще отклонять $H_0$, когда отличий нет).

Посчитаем количество значимых отличий в каждой группе после того, как произведена коррекция Бенджамини-Хохберга, причем так же, как и во второй части, считаем только такие отличия, у которых abs(fold change) $> 1.5$. 

In [218]:
reject1, p_corrected1, a1, a2 = smm.multipletests(pvalues1,
                                                  alpha=0.05/2,
                                                  method='fdr_bh')

mask = np.array(p_corrected1) < 0.025

In [219]:
fold_changes = fold_change(normal_mean[mask], early_neoplasia_mean[mask])

ans5 = sum(abs(fold_changes) > 1.5)

write_ans(ans5, answer_number=5)

print("Answer",ans5)

Answer 4


In [220]:
reject, p_corrected2, a1, a2 = smm.multipletests(pvalues2,
                                                 alpha=0.025,
                                                 method='fdr_bh')
mask = np.array(p_corrected2) < 0.025

In [222]:
fold_changes = fold_change(early_neoplasia_mean[mask], cancer_mean[mask])

ans6 = sum(abs(fold_changes) > 1.5)

write_ans(ans6, answer_number=6)

print("Answer", ans6)

Answer 524
